In [1]:
import random

In [18]:

def generate_balanced_brackets(n, max_depth):
    brackets = ['(', ')']
    map = {'(': ')'}
    sequence = ''
    stack = []
    
    for i in range(n):
        open_or_close = random.choice([0, 1]) and stack
        if n-i < len(stack) or (not open_or_close and len(stack) == n-i) or len(stack) == max_depth:
            open_or_close = 1
        if open_or_close:
            sequence += map[stack.pop()]
        else:
            bracket = '('
            sequence += bracket
            if bracket in map:
                stack.append(bracket)
    
    return sequence

def is_balanced(s):
    stack = []
    brackets = {')': '('}
    max_depth = 0
    
    for bracket in s:
        if bracket in brackets:
            if stack:
                stack.pop()
            else :
                break
        else:
            stack.append(bracket)
            max_depth = max(max_depth, len(stack))
    
    return (-1*max_depth) if stack else max_depth

def generate_bracket_sequence(n):
    brackets = ['(', ')']
    sequence = ''
    
    for _ in range(n):
        sequence += random.choice(brackets)
    # print('Sequence:', sequence)
    
    if is_balanced(sequence) > 0:
        sequence = generate_bracket_sequence(n)
    
    return sequence

In [3]:
import pandas as pd

# Generate 2000 balanced sequences
balanced_sequences = [generate_balanced_brackets(64) for _ in range(2000)]

# Generate 10000 unbalanced sequences
unbalanced_sequences = [generate_bracket_sequence(64) for _ in range(10000)]

# Create the dataframe
df = pd.DataFrame({'Sequence': balanced_sequences + unbalanced_sequences, 'Balanced': [1] * 2000 + [0] * 10000}, columns=['Sequence', 'Balanced'])

# shuffle the dataframe
df = df.sample(frac=1).reset_index(drop=True)

# Print the dataframe
print(df.head())

TypeError: generate_balanced_brackets() missing 1 required positional argument: 'max_depth'

In [ ]:
df.to_csv('balanced_brackets.csv', index=False)

In [ ]:
generate_balanced_brackets(1000, 100)

'()()()(())()((()))()(()()()())(()())()()()((()()()((())(((())))(()((())((()(()()))()))((((((()()())()((((()()())))()(((())()()()())(((())()())(((())(((()()(())))))()()))(()(()))((()))())()((())((()())((()(()(((()()((())))()((((()())()(()))))))))(()))(((()((()()((((())()()()(()()((((()((((())()))((((())((((()()((()((()))(((((()()((())((()(((())((((((())(()()()((())))())()))()())())())(((())()))))(((((()))(())((((())))(()()(()))()()()())()(((()())((()(())())())((())()(()))))(())()()((())()))(()(())(((()(()())())(()))()(())(((((()()((()(()((((()())()())))((()())))((((((())()(()()(()))(()()()()))()(())))))()(((()(()((()(()()()())))))))((())())))))())(())))())))))))()))()))))())(())()))(()()))()((()()))((()))()())(())))))(()))))(()(((()()))))))((((()(()(((()(())(()()((()()))(((()()())(((())(()))())()))((()))()))())))())))(()((())))()()))(((((()))(()())((())))(((()(()()())(()(((((()()(())))))))()())())())))()))(((((())()(()((((())(()(()(()()(((())((((()()())())))))))))))))))))))))))))))))))

In [19]:
import pandas as pd


balanced_df = pd.DataFrame(columns=['sequence', 'stack_depth'])


for i in range(1,200):
    for j in range(1,i+1):
        sequence = generate_balanced_brackets(i*2, j)
        balanced = is_balanced(sequence)
        
        # bracket_data = bracket_data.append({'sequence': sequence, 'balanced': balanced != -1, 'stack_depth': balanced}, ignore_index=True)
        # append row to df
        balanced_df.loc[len(balanced_df)] = [sequence, balanced]

In [20]:
unbalanced_data = [ generate_bracket_sequence(random.randint(1, 200)) for _ in range(balanced_df.shape[0]) ]
unbalanced_stack_depth = [ is_balanced(sequence) for sequence in unbalanced_data ]

unbalanced_stack_depth

IndexError: pop from empty list

In [ ]:

unbalanced_df = pd.DataFrame({'sequence': unbalanced_data, 'stack_depth': unbalanced_stack_depth}, columns=['sequence', 'stack_depth'])

In [9]:
bracket_data = pd.concat([balanced_df, unbalanced_df], ignore_index=True)

bracket_data = bracket_data.sample(frac=1).reset_index(drop=True)

In [10]:
bracket_data    

,sequence,stack_depth
0,()()((()))(()((()()()()())(())))(())()(())()((...,34
1,()()()()()((()(()))())()(())(())((((()()((()()...,11
2,)()))()()(())(((())))())))(()))((()()(()(())()...,False
3,()())()((()(())))(()())((()())))()(())))((((()...,False
4,()()()(()()()((()(((()))(((((((())))(()(((())(...,41
...,...,...
39795,))(()()(()))((()()))()())))(())()())))((((()()),False
39796,()()()(()((((())((()(())()(()))((((()(()(((()(...,20
39797,()()(())(()(((((())())))((()()(())))()))()((((...,10
39798,()()(()()((())()))()(()(()(())(())()((())()())...,16


In [ ]:
bracket_data.to_csv('balanced_brackets.csv', index=False)